In [88]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [109]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import sys
import urllib.request
import urllib.parse
import json
import requests

#selenium의 webdriver를 사용
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

### OPENAI, kakao rest api, naver api 키를 .env 파일로부터 load

In [59]:
# env 파일 로드
load_dotenv()

# openai api, kakao rest api, naver api 키 로드
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
KAKAO_REST_API_KEY = os.getenv('KAKAO_RESTAPI_KEY')
NAVER_API_CLIENT_ID = os.getenv('NAVER_API_CLIENT_ID')
NAVER_API_CLIENT_SECRET = os.getenv('NAVER_API_CLIENT_SECRET')

# 프롬프트
prompt = """지금부터 당신은 장소 리뷰어가 된다. 
아래 스팟 리스트에 대해서 google에 해당 스팟의 리뷰에 대해, 부가적인 내용없이 각 스팟에 대해 출력 형식에 맞춰 답변한다.

스팟 리스트
"""

prompt2 = """
출력 형식
장소 이름/긍정평가:n%/부정평가:n%
출처: url
"""

### OpenAI api를 활용해 ChatGPT 활용하기

In [39]:
def openai_message(message):
    client = OpenAI(api_key = openai_api_key)
    completion = client.chat.completions.create(
    model="gpt-4o-mini-search-preview",
    web_search_options={
        "user_location": {
            "type": "approximate",
            "approximate": {
                "country": "KR",  # 대한민국
                "city": "Chuncheon",  # 춘천
                "region": "Gangwon"  # 강원도
            }
        },
        "search_context_size": "medium"  # 검색 컨텍스트 크기 (low, medium, high 중 선택)
    },
    messages=[{
        "role": "user",
        "content": message
    }]
)

    print(completion.choices[0].message.content)

In [58]:
def kakao_rest_api(longitude, latitude):
    url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json'
    headers = {'Authorization': f'KakaoAK {KAKAO_REST_API_KEY}'}
    params = {'x': longitude, 'y': latitude}
    
    response = requests.get(url, headers=headers, params=params)
    
    # 데이터를 성공적으로 불러온 경우, code = 200
    if response.status_code == 200:
        return response.json()
    else:
        return None

### 카카오 Rest api를 활용해 해당 좌표의 카테고리별 결과 불러오기

In [57]:
def search_by_category(longitude, latitude, category_code):
    url = 'https://dapi.kakao.com/v2/local/search/category.json'
    headers = {'Authorization': f'KakaoAK {KAKAO_REST_API_KEY}'}
    params = {
        'category_group_code': category_code,
        'x': longitude,
        'y': latitude,
        'radius': 1000,  # 검색 반경 설정 (단위: 미터)
        'size': 15       # 한 페이지에 보여질 결과 개수
    }

    # API 요청
    response = requests.get(url, headers=headers, params=params)

    # 응답 처리
    if response.status_code == 200:
        return response.json()  # 성공적으로 데이터를 반환
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None  # 실패 시 None 반환

### 네이버 API를 통한 키워드 검색

In [78]:
def search_naver_blog(keyword):
    client_id = NAVER_API_CLIENT_ID
    client_secret = NAVER_API_CLIENT_SECRET
    # 블로그에 검색할 키워드
    encText = urllib.parse.quote(keyword)
    url = "https://openapi.naver.com/v1/search/blog?query=" + encText
    
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        return json.loads(response_body.decode('utf-8'))
    else:
        print("Error Code:" + rescode)

### 네이버 API로 주어진 키워드에 맞는 블로그 링크를 가져오기

In [111]:
# API 호출 및 데이터 처리
result = search_naver_blog("리네 춘천")

# 네이버 API로 주어진 키워드에 맞는 블로그 링크를 가져오기
if result is not None:
    link = [item['link'] for item in result.get('items', [])]
    for url in link:
        print(url)
        print(get_naver_blog_content(url))
else:
    print("API 요청에 실패했습니다")

https://blog.naver.com/sein0728/223551485543
본문: 
 돼지갈비 ₩14,000*2
된장찌개 ₩3,000
공기밥 ₩1,000
빈이가 그렇게 극찬했던 육림 숯불구이 집에 왔습니다.
양이 진짜 많다고 했는데 ㄹㅇ 배터질뻔 했어요.
된찌도 집에서 담구신 장으로 하신 것 같았고, 김치가 너무너무 맛있었던 집
배부른 와중에 하나도 안남기고 다 먹고 왔어요.
왜이렇게 극찬을 했는지 알정도
춘천 뜨기 전에 한번 더 간다!! 겨울방학에 무조건 한번 더 가야지!!
육림보쌈
강원특별자치도 춘천시 소전길 33-1
요번에는 강대병원에 갔다가 오는길에 먹었던 리네!
춘천에서 우리한테 외식은 항상 리네 였는데
이번에는 일을 하고 있었던터라 오랜만에 왔어요.
빈이한테는 간간히 구성이 변했다는 소식은 들었지만
그래도 처음먹었을때의 그 느낌이 너무 진해서 한번 더 왔습니다.
카에라멘 ₩7,500
이게 제 쵀애 라면입니다.
음 확실히 구성이 바뀌긴했어요.
원래 없던 감자도 생기고 새우도 원래 껍질째 있는 큰새우 두개?에서 껍질이 까져있는 새우로
꽃개는 그대로 두개였구요.
원래는 콩 입자도 안보였던 것 같은데 보였구, 무엇보다 면이 바뀐 느낌? 원래 이렇게 굵지는 않았던 것 같은데 쫄깃함 보다는 끊어지는 면으로 된 것 같아요.
근데 그걸 다 이기는 국물 맛 된장의 맛과 갑각류로 낸 국물의 맛이 진짜 천상의 맛
밥이 땡기는데 면도 땡기는 그 맛
새우를 바꾸셔서 그런지 전보다는 덜했지만 그래도 사장님이 대학가 상권이다 보니깐 가격을 유지하되 내용물을 바꾸신 선택을 하신것 같아서 이해는 된 부분이에요.
이 가격에 이 퀄리티면 전 계속 먹을 것 같거든요.
그래도 원래 맛을 프리미엄 가격을 붙여서 파시면 안될까요..? 소량으로 라도… 그맛 진짜 맛있었는데
맞다 이 날은 자리가 없어서 2층에서 먹었습니다.
그만큼 인기도 많은 집이라는거!
신기돈 ₩9,500
요건 빈이꺼 빈이도 학교다니면서 친구들한테 소개해주다가 한친구가 이것만 시켜먹어서 본인도 빠졌다고 한다.
밥(소) 

In [101]:
def get_naver_blog_content(url):
    # 본문 내용을 저장할 리스트
    contents = []
    
    # Selenium WebDriver 설정하기
    driver = webdriver.Chrome()

    try:
        driver.get(url)
        # iframe 로딩 대기 및 전환
        WebDriverWait(driver, 10).until(
            EC.frame_to_be_available_and_switch_to_it((By.ID, "mainFrame"))
        )

        #본문 내용 크롤링하기
        try:
            # 최신 네이버 블로그
            a = driver.find_element(By.CSS_SELECTOR,'div.se-main-container').text
            contents.append(a)
            
        except NoSuchElementException:
            # 구 버전 네이버 블로그
            a = driver.find_element(By.CSS_SELECTOR,'div#content-area').text
            contents.append(a)
        print('본문: \n', a)
        
    except Exception as e:
        print(f"오류 발생: {e}")
        
    finally:
        driver.quit() #창닫기
        print("<<본문 크롤링이 완료되었습니다.>>")

    return contents

In [46]:
# 강원대학교 후문의 경도와 위도
longitude = "127.73983"  # 경도
latitude = "37.87760"    # 위도
category_code = "CE7"    # 카테고리 코드 (예: 카페)


# 프롬프트 초기화
prompt = """지금부터 당신은 장소 리뷰어가 된다. 
아래 스팟 리스트에 대해서 google에 해당 스팟의 리뷰에 대해, 부가적인 내용없이 각 스팟에 대해 출력 형식에 맞춰 답변한다.

스팟 리스트
"""

prompt2 = """
출력 형식
장소 이름/긍정평가:n%/부정평가:n%
출처: url
"""

# API 호출 및 데이터 처리
result = search_by_category(longitude, latitude, category_code)

if result is not None:  # API 요청 성공 여부 확인
    documents = result.get('documents', [])
    if documents:  # 검색 결과가 있을 경우
        for idx, place in enumerate(documents, start=1):
            name = place.get('place_name')       # 장소 이름
            address = place.get('address_name')  # 주소
            
            prompt += str(idx) + ". " + name + ", " + address + "\n"
    else:  # 검색 결과가 없을 경우
        prompt += "검색 결과가 없습니다.\n"
else:
    print("API 요청에 실패했습니다.")

# 프롬프트 완성
prompt += prompt2

print(prompt)

지금부터 당신은 장소 리뷰어가 된다. 
아래 스팟 리스트에 대해서 google에 해당 스팟의 리뷰에 대해, 부가적인 내용없이 각 스팟에 대해 출력 형식에 맞춰 답변한다.

스팟 리스트
1. 그라시아커피로스터스, 강원특별자치도 춘천시 교동 148-4
2. 포지티브즈, 강원특별자치도 춘천시 효자동 619-1
3. 예담카페, 강원특별자치도 춘천시 효자동 601-11
4. 그빵집비비, 강원특별자치도 춘천시 운교동 186-8
5. 교토정원, 강원특별자치도 춘천시 조양동 6-115
6. 아글라오네마, 강원특별자치도 춘천시 후평동 689-3
7. 소셜하우스, 강원특별자치도 춘천시 효자동 758-12
8. 퍼스트러브, 강원특별자치도 춘천시 효자동 607-8
9. 오르미, 강원특별자치도 춘천시 효자동 745-6
10. 메가MGC커피 춘천팔호광장점, 강원특별자치도 춘천시 효자동 653
11. 스타벅스 강원대점, 강원특별자치도 춘천시 효자동 632-3
12. 카페위위, 강원특별자치도 춘천시 효자동 623-8
13. 네모커피, 강원특별자치도 춘천시 효자동 753-8
14. 책방마실, 강원특별자치도 춘천시 옥천동 39-21
15. 핀든하우스, 강원특별자치도 춘천시 운교동 134-9

출력 형식
장소 이름/긍정평가:n%/부정평가:n%
출처: url

죄송하지만, 제공된 장소들의 최신 리뷰 정보를 찾을 수 없었습니다. 리뷰 정보는 시간이 지남에 따라 변동될 수 있으므로, 각 장소의 최신 리뷰를 확인하시려면 구글 지도나 해당 장소의 공식 웹사이트를 방문하시는 것을 권장드립니다. 


In [ ]:
pip install selenium